In [ ]:
# We need this for creating directories\
import os
from pathlib import Path

# For session info
import session_info

# We need this for downloading data from refine.bio
import pyrefinebio

# We will use these for manipulating our data
import statistics
import pandas as pd
import numpy as np
import random

# We will use this for making the heatmap and these for the color codes
import seaborn as sns
sns.set_theme(color_codes=True)

# Import custom color key function
import util.color_key as color_key

import matplotlib.pyplot as plt

In [ ]:
random.seed(1234)

In [ ]:
dirnames = ["data", "results", "plots"]

for dirname in dirnames:
    if not os.path.isdir(dirname):
        os.mkdir(dirname)

In [ ]:
id = "SRP070849"

In [ ]:
data_dir = Path(f"data/{id}")
data_file = data_dir.joinpath(f"{id}.tsv")
metadata_file = data_dir.joinpath(f"metadata_{id}.tsv")

In [ ]:
if not data_file.exists() or not metadata_file.exists():
    print(f"Downloading {id} from refine.bio")
    pyrefinebio.create_token(agree_to_terms=True, save_token=False)
    pyrefinebio.download_dataset(
        data_dir.joinpath("dataset.zip"),
        "cansav09@gmail.com",
        experiments=[f"{id}"],
        extract=True,
    )

In [ ]:
metadata = pd.read_csv(metadata_file, sep="\t")

In [ ]:
expression_df = pd.read_csv(data_file, sep="\t")

In [ ]:
expression_df.set_index("Gene", inplace=True)

In [ ]:
metadata.head(5)

In [ ]:
print(metadata["refinebio_accession_code"].tolist() == expression_df.columns.tolist())

In [ ]:
# Calculate the variance for each gene
expression_df["variance"] = expression_df.var(axis=1, skipna=True)

# Find the upper quartile for these data
upper_quartile = expression_df["variance"].quantile([0.90]).values

# Filter the data choosing only genes whose variances are in the upper quartile
df_by_var = expression_df[expression_df.variance > float(upper_quartile)]

# Drop the variance column we calculated
df_by_var = df_by_var.drop(columns="variance")

In [ ]:
# Check how many genes we are left with
print(len(df_by_var.index))

In [ ]:
# Write df_by_var to tsv file
df_by_var.to_csv("results/top_90_var_genes.tsv", sep="\t")

In [ ]:
# Recode refinebio_title variable into something more useful
# Split wherever there is a '-' or '.'
exp_group = metadata["refinebio_title"].str.split("-|\\.", expand=True, n=1)

In [ ]:
# Now we'll store that first column from the string split back in the metadata as a column
metadata["exp_group"] = exp_group[0]

In [ ]:
# Print to check
print(metadata["exp_group"])

In [ ]:
# Color code our two variables
refinebio_treatment_colors = color_key.make_color_key(metadata["refinebio_treatment"])
exp_groups_colors = color_key.make_color_key(metadata["exp_group"])

In [ ]:
color_key_df = pd.concat(
    [refinebio_treatment_colors["color_key"], exp_groups_colors["color_key"]],
    axis=1,
    names=["treatment", "exp_group"],
)

color_key_df = color_key_df.set_index(df_by_var.columns)

In [ ]:
# Check that this is what we think it is
color_key_df

In [ ]:
heatmap = sns.clustermap(
    df_by_var,
    cmap="mako",
    col_colors=color_key_df,
    dendrogram_ratio=(0, 0.2),
    cbar_pos=(-0.1, 0.2, 0.03, 0.5),
)

legend1 = color_key.make_legend(exp_groups_colors["color_key_dict"])
legend2 = color_key.make_legend(refinebio_treatment_colors["color_key_dict"])

# Make a legend 1
plt.legend(
    legend1,
    exp_groups_colors["color_key_dict"],
    title="Experiment \nGroup",
    bbox_to_anchor=(4, 1.3),
)

# Make a legend 2
heatmap.ax_row_dendrogram.legend(
    legend2,
    refinebio_treatment_colors["color_key_dict"],
    title="Treatment",
    bbox_to_anchor=(0, 1.37),
)

plt.show()

In [ ]:
# Save to png
heatmap.savefig(os.path.join("plots", "aml_heatmap.png"))

In [ ]:
session_info.show()